In [ ]:
#export
from IPython.display import Latex

In [ ]:
#export
def makeVar(*args):
    if len(args) == 1: 
        return Var(str(args[0])) if not isinstance(args[0], Var) else args[0]
    vs = []
    for arg in args:
        vs.append(Var(str(arg))) if not isinstance(arg, Var) else vs.append(arg)
    return vs

In [ ]:
#export
class Var():
    def __init__(self, name, latex, isRoot=True):
        self._name = str(name)
        self._fn = None
        self._fn, self._isRoot = latex, isRoot
        
    @property
    def name(self): return self._name
    @name.setter
    def name(self, n): self._name = n
    def setName(self, n): 
        self._name = n
        return self
    
    def shouldForceEnv(self):
        return self._isRoot
    def removeRootStatus(self):
        self._isRoot = False
        
    @property
    def back(self): return self._fn
    @back.setter
    def back(self, fn): self._fn = fn
      
    def __str__(self): return self._name
    def __repr__(self): return self.__str__()
    
    def apply(self, f, *args):
        n = Var('temp')
        n.back = f(*args)
        return n
    
    def __add__(self, other):
        return self.apply(BinaryFunc, ' + ', self, other)
    def __sub__(self, other):
        return self.apply(BinaryFunc, ' - ', self, other)
    def __mul__(self, other):
        return self.apply(BinaryFunc, ' * ', self, other)
    def __xor__(self, other):
        return self.apply(BinaryFunc, '^', self, other)
    def __pow__(self, other):
        return self.apply(BinaryFunc, '^', self, other)
    def equals(self, other):
        return self.apply(BinaryFunc, ' = ', self, other)
    def __or__(self, other):
        return self.apply(BinaryFunc, r'\hspace{1mm}', self, other)
    
    def root(self, other):
        return self.apply(RootFunc, self, other)
    def __truediv__(self, other):
        return self.apply(FracFunc, self, other)
    
    def build(self):
        return self.name if self._fn is None else self._fn.build()
        
    def func_appl(self, *others):
        return self.apply(FuncApplication, self, *others)
    def __call__(self, *others):
        return self.apply(FuncApplication, self, *others)
    
def MultiVar(*args):
    return [Var(ele) for ele in args]

def addEnv(s, env='gather'):
    env = '{' + env + '}'
    return r'\begin' + env + str(s) + r'\end' + env

def func(f, *args):
    f = makeVar(f)
    args = makeVar(*args)
    return f.func_appl(*args)

def wrap(v):
    return Latex(addEnv(v.build()))

In [ ]:
#export
class BinaryFunc():
    def __init__(self, mid, a1, a2):
        self._mid = mid
        self.a1, self.a2 = makeVar(a1, a2)
    def build(self):
        if self._mid is None:
            raise Exception('middle operator string not set')
        return f'{self.a1.build()}{self._mid}{self.a2.build()}'
                
class FracFunc():
    def __init__(self, num, den):
        self.num, self.den = makeVar(num, den)
    def build(self):
        return (r"\frac{" + 
                self.num.build() + r"}{" + self.den.build() + r"}")
class RootFunc():
    def __init__(self, num, root):
        self.num, self.root = makeVar(num, root)
    def build(self):
        return(r"\sqrt[\leftroot{-1}\uproot{1}" + self.root.build() 
               + r"]{" + self.num.build() + r"}")
    
class IntegralFunc():
    def __init__(self, lower, upper, func):
        self.lower, self.upper, self.func = lower, upper, func
    def build(self):
        pass
    
class FuncApplication():
    def __init__(self, f, *args):
        self.f = makeVar(f)
        if len(args) == 0: self.args = None
        else: 
            self.args = makeVar(*args)
            if not isinstance(self.args, list): self.args = [self.args]
        
    def build(self):
        if self.args is None: return self.f.build() + '()'
        s = self.f.build() + '('
        for arg in self.args:
            s += arg.build() + ','
        s = s[:-1]
        return s + ')'

In [ ]:
#export
class Vector(Var):
    def __init__(self, name):
        super().__init__(r"\vec{\mathbf{" + name +"}}")

In [ ]:
#export
class MatrixBuilder():
    def __init__(self):
        self.matrix, self.cols = [], None
        
    def add(self, *args):
        if self.cols is None:
            self.cols = len(args)
        elif not self.cols == len(args):
            raise Exception(f'Different dimensions for matrix builder. dim1:{self.cols} dim2:{len(args)}')
        self.matrix.append(list(args))
        return self
        
    def create(self, subscript=None, power=None, surround='[]'):
        return Matrix(self.matrix, subscript, power, surround)

In [ ]:
#export
class Matrix(Var):
    def __init__(self, matrix=[[]], subscript=None, power=None, surround='[]'):
        super().__init__('matrix')
        self.matrix, self.surround = matrix, surround
        self.power = makeVar(power) if power is not None else power
        self.subscript = makeVar(subscript) if subscript is not None else subscript
        
    def __len__(self):
        return self.rows
    
    def __getitem__(self, ind):
        if ind  >= len(self.matrix): raise Exception('index out of bounds')
        if len(self.matrix) == 0: raise Exception('empty matrix') 
        if isinstance(self.matrix[0], list): return Matrix(self.matrix[ind])
        else: return Var(self.matrix[ind])
    
    @staticmethod
    def builder():
        return MatrixBuilder()
        
    @property
    def T(self):
        if len(self.matrix) == 0: return self
        elif(not isinstance(self.matrix[0], list)):
            rows, cols = len(self.matrix), 1
            matrix = [[ele] for ele in self.matrix]
        else:
            rows, cols = len(self.matrix), len(self.matrix[0])
            matrix = [[self.matrix[j][i] for j in range(rows)] for i in range(cols)]
        return Matrix(matrix, self.subscript, self.power, self.surround)
        
    def build(self): 
        env = (r'{bmatrix}' if self.surround == '[]' 
               else r'{pmatrix}' if self.surround == '()'
               else r'{matrix}')
        begin = r'\begin' + env
        end = r'\end' + env
        if len(self.matrix) == 0: return begin + end
        
        s = ''
        if not isinstance(self.matrix[0], list):
            # it is a vector
            for ele in self.matrix:
                ele = makeVar(ele)
                s += ' ' + ele.build() + ' & '
            s = s[:-2]
        else:
            for row in self.matrix:
                for ele in row:
                    ele = makeVar(ele)
                    s += ' ' + ele.build() + ' & '
                s = s[:-2] + r"\\"
            s = s[:-2]
        p = '^{' + self.power.build() + '}' if self.power is not None else ''
        subs = '_{' + self.subscript.build() + '}' if self.subscript is not None else ''
        return (r"" + begin + s + end  + p + subs + r"")


In [ ]:
#export
class Dots(Var):
    def __init__(self, direc='h'):
        super().__init__(direc + 'dots')
        self.direc = direc
    def build(self):
        return (r'\cdots' if (self.direc == 'h' or self.direc == 'c') 
                else r'\vdots' if (self.direc == 'v') else r'\ddots')

In [ ]:
#export
def hasHoriDots(elements, m):
    elerowlen = max([len(ele) for ele in elements]) 
    return not elerowlen == len(m[0])
def hasVertDots(elements, m):
    return not len(elements) == len(m)

class MatrixWithDots(Matrix):
    def __init__(self, elements, shape=None, subscript=None, power=None, surround='[]'):
        # resolve elements
        self.elements, self.shape = elements,shape
        if(not isinstance(elements, list) or len(elements) == 0
          or not isinstance(elements[0], list)): 
            raise Exception('need to pass list of list')
        # resolve shape
        minrows, mincols = len(elements), max([len(row) for row in elements])
        if(shape is None):
            self.rows = minrows + 1 if minrows != 1 else minrows
            self.cols = mincols + 1 if mincols != 1 else mincols
        else:
            try:
                self.rows, self.cols = shape
                self.rows, self.cols = int(self.rows), int(self.cols)
            except:
                raise Exception('shape should be of type tuple(int,int)')
            if self.rows < minrows or self.cols < mincols:
                raise Exception('shape >= (row, cols) of elements')
        m = [[0]*self.cols for i in range(self.rows)]
        i=0
        while i < len(elements)-1:
            row = elements[i]
            j=0
            while(j < len(row)-1):
                m[i][j] = row[j]
                j += 1
            while(j < len(m[i])-1):
                m[i][j] = Dots('h')
                j += 1
            m[i][j] = row[len(row)-1]
            i += 1

        while i < len(m)-1:
            m[i] = [Dots('v')]*len(m[i])
            i += 1

        row = elements[len(elements)-1]
        j=0
        while(j < len(row)-1):
            m[i][j] = row[j]
            j += 1
        while(j < len(m[i])-1):
            m[i][j] = Dots('h')
            j += 1
        m[i][j] = row[len(row)-1]

        dli, dlj = len(m)-2, len(m[0])-2

        if isinstance(m[dli][dlj], Dots) and hasHoriDots(elements, m) and hasVertDots(elements, m):
            m[dli][dlj] = Dots('d')
        super().__init__(m, subscript, power, surround)
        
    @property
    def T(self):
        raise Exception('transpose of dotted matrix not allowed')

In [ ]:
wrap(MatrixWithDots([[1],[2],[2]], (3,1)))

In [ ]:
wrap(MatrixWithDots([[1,2,3,4],[2,3,4,5]]))

In [ ]:
!./notebook2script.py ./Variables.ipynb